In [ ]:
!wget "https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta"

In [1]:
def loadFasta(filename):
    f = open(filename)
    name = ''
    dna = ''
    
    for line in f:
        if line[0] == '>':
            name = line[1:].rstrip()
            
        else:
            dna += line.rstrip()
    return name, dna

In [2]:
_, genome = loadFasta('chr1.GRCh38.excerpt.fasta')

In [3]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    distance = D[-1]
    distance.sort()
    return distance[0]

In [4]:
# question 1 
p = 'GCTGATCGATCGTACG'
result = editDistance(p,genome)
print(result)

3


In [5]:
# question 2 
p = 'GATTTACCAGATTGAG'
result = editDistance(p,genome)
print(result)

2


In [6]:
# question3 

def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [7]:
from itertools import permutations
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [29]:
def readFASTQ(filename):    
    f = open(filename)
    reads = []
    while True:
        h = f.readline()
        if len(h) == 0:
            break
        sequence = f.readline().rstrip()
        _ = f.readline()
        quality = f.readline().rstrip()
        reads.append(sequence)
        
    return reads

In [18]:
!wget "https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq"

--2024-07-06 14:06:53--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.154.95, 143.204.154.101, 143.204.154.225, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.154.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  10.2MB/s    in 0.2s    

2024-07-06 14:06:54 (10.2 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [9]:
file = 'ERR266411_1.for_asm.fastq'
reads = readFASTQ(file)

In [10]:
overlaps = naive_overlap_map(reads, 30)

In [11]:
print(len(overlaps))

904746


In [12]:
s = set()
for a, b in overlaps.keys():
    s.add(a)

In [13]:
len(s)

7161

## module 4 

In [4]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [44]:
# question 1
c = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
short_common_str = scs(c)
print(short_common_str, len(short_common_str))


CCTTGGATTGC 11


In [53]:
# question 2
def scs_all(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    lst_ssc = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) <= len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
            lst_ssc.append(shortest_sup)
    mini_lst = [i for i in lst_ssc if len(i) == len(min(lst_ssc))]
    
    return mini_lst # return shortest

In [47]:
strings = ['ABC', 'BCA', 'CAB']
scs_all(strings)

['ABCAB', 'BCABC', 'CABCA']

In [48]:

strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']
scs_all(strings)

['TCGATGCAATAG',
 'TCGATAGAATGC',
 'TCGAATAGATGC',
 'TGCAATCGATAG',
 'TGCAATAGATCG',
 'AATCGATAGTGC',
 'AATGCTCGATAG',
 'AATAGATCGTGC',
 'AATAGATGCTCG',
 'AATAGTCGATGC']

In [54]:
# answer for 2 
c = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
scs_all(c)

['CCTTGCTGGATT', 'CCTTGCGATTGG']

In [51]:
scs_all(c)

{'CCTTGGATTGC', 'GATTGCCTTGG', 'TGCCTTGGATT', 'TGGATTGCCTT'}

In [24]:
# question 3 

!wget "https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq"

--2024-07-06 19:31:25--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.154.95, 143.204.154.225, 143.204.154.101, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.154.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395781 (387K) [video/m2ts]
Saving to: ‘ads1_week4_reads.fq’

ads1_week4_reads.fq 100%[===================>] 386.50K  --.-KB/s    in 0.1s    

2024-07-06 19:31:26 (3.69 MB/s) - ‘ads1_week4_reads.fq’ saved [395781/395781]



In [30]:
file = readFASTQ('ads1_week4_reads.fq')

['GTCCAGCAGAGCAAGTGATGCGAGAGCTGCCCATCCTCCAACCAGCATGCCCCTAGACATTGACACTGCATCGGAGTCAGGCCAAGATCCGCAGGACAGT',
 'GGAGTACGACTTCAGAGATCTCACTTGGTGTATCAACCCGCCAGAGAGAATCAAATTGGATTATGATCAATACTGTGCAGATGTGGCTGCTGAAGAACTC',
 'GCAAATTTTGATCTCTCTTGGCTTCACAATCAATTCAACCATGACCCGAGATGTAGTCATACCCCTCCTCACAAACAACGATCTCTTAATAAGGATGGCA',
 'GAGTTAATTGAAGCCCTAGATTACATTTTCATAACTGATGACATACATCTGACAGGGGAGATTTTCTCATTTTTCAGAAGTTTCGGCCACCCCAGACTTG',
 'AATGACAGAGACCGCTATGACCATTGATGCTAGGTATGCAGAACTTCTAGGAAGAGTCAGATACATGTGGAAACTGATAGATGGTTTCTTCCCTGCACTC',
 'TACGATTTCGACAAGTCGGCATGGGACATCAAAGGGTCGATCGCTCCGATACAACCTACCACCTACAGTGATGGCAGGCTGGTGCCCCAGGTCAGAGTCA',
 'AGAGGAGGCAGGCAGTTCGGGTCTCAGCAAACCATGCCTCTCAGCAATTGGATCAACTGAAGGCGGTGCACCTCGCATCCGCGGTCAGGGATCTGGAGAA',
 'CTCTCATCTCACAGAGGTGTTATCGCTGACAATCAAGCAAAATGGGCTGTCCCGACAACACGGACAGATGACAAGTTGCGAATGGAGACATGCTTCCAGC',
 'GTCAGGTTAATTGGAAACCCGGATGTGAGCGGGCCCAAACTAACAGGGGCACTAATAGGTATATTATCCTTATTTGTGGAGTCTCCAGGTCAATTGATTC',
 'CTAACACGGTATTACATCTTCACGTCGAAACAGATTGTTGCGTGATCCCGATG

In [32]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    
    kmer_dict = {}
    
    for read in reads:
        for i in range(len(read)-k+1):
            kmer_dict[read[i:i+k]] = set()
            
    for read in reads:
        for i in range(len(read)-k+1):
            kmer_dict[read[i:i+k]].add(read)
   
    for read in reads:
        current_kmer_set = kmer_dict[read[-1*k:]]
        for kmer_read in current_kmer_set:
            if read != kmer_read:
                olen = overlap(read, kmer_read, min_length=k)                
                if olen > best_olen:
                    reada, readb = read, kmer_read
                    best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)


In [34]:
result = greedy_scs(file, 10)

In [35]:
print(len(result)) # we matched 

15894


In [36]:
lst_A = [i for i in result if i == 'A']
print(len(lst_A))

4633


In [37]:
# question 4 
lst_T = [i for i in result if i == 'T']
print(len(lst_T))

3723
